This script provides a framework for building a CI pipeline.

Jarvis seems like a natural combination between Jenkins and Travis.

In [8]:
import re

In [9]:
def update_kitchen_yml(lines, prev_pattern, pattern, repl):
    """Perform a two-line search to update the correct versions in kitchen.yml."""
    # Safe to setup first two lines in a multiline match.
    prev = next(lines)
    for line in lines:
        curr = line
        if re.search(prev_pattern, line) and re.search(pattern, line):
            # Update on match
            yield re.sub(pattern, repl, line, count=0)
        prev = curr

In [65]:
source = '''
  attributes:
    dyn:
      hostname: "zephyr24-00-dev"
      site: "dev"
    dyn_zephyr:
      env: dev
      vangmeh:
        version: 111.204.43
      majqa:
        version: 1.0.5

'''
patterns = [
    re.compile(r'''
    ^\s+vangmeh:$
    ''', re.X | re.M ),
    re.compile(r'(\d\.?){3}'),
    re.compile(r'''
(\s+vangmeh:\n
\s+version:\s)(?P<version>(\d\.?){3})
''', re.X)
]

for p in patterns:
    m = p.search(source)
    if m:
        print(m.group(0))
    else:
        print("No match found")
        
s = patterns[-1].sub('\g<1>1.1.0', source, 1)
if s:
    print(s)

      vangmeh:
111.

      vangmeh:
        version: 111.

  attributes:
    dyn:
      hostname: "zephyr24-00-dev"
      site: "dev"
    dyn_zephyr:
      env: dev
      vangmeh:
        version: 1.1.0204.43
      majqa:
        version: 1.0.5




In [76]:
def read_until_valid(prompt, valid_inputs=None, lmbda=None):
    """Loop until a valid input has been received.

    It is up to the caller to handle exceptions that occur outside the realm of
    calling their lambda, such as KeyboardInterrupts (^c, a.k.a C-c).

    :arg str prompt: Prompt to display.
    :kwarg ``Iterable`` valid_inputs: Acceptable inputs. If none are provided,
        then the first non-exceptional value entered will be returned.
    :arg ``func`` lmbda: Function to call on received inputs. Any errors will
        result in a re-prompting.
    """
    while True:
        user_input = input(prompt).strip(string.whitespace)
        # Apply a given function
        if lmbda is not None:
            try:
                user_input = lmbda(user_input)
            except:         # Any errors are assumed to be bad input
                continue    # So keep trying
        if valid_inputs is not None:
            if user_input in valid_inputs:
                return user_input
        else:
            return user_input


def bump_kitchen_yml(source, name, new_version, prompt=True):
    """Performs a version bump on .kitchen.yml file.
    
    :arg str source: .kitchen.yml file read into memory.
    :arg str name: Name of target binary to bump
    :arg str new_version: Version to bump to.
    """
    # Master string template for .kitchen.yml version attributes
    re_string = '''
(\s+{name}:\n
\s+version:\s+)(?P<version>(\d+\.?){{3}})
'''.format(name=name)
    p = re.compile(re_string, re.X)
    
    # Preview the change to our user
    m = p.search(source)
    print('Bumping version from {old} => {new}'.format(old=m.group('version'), new=new_version))
    print('Preview:{body}{new}'.format(body=m.group(1), new=new_version))
    
    # Request permission to continue (unless told not to)
    if prompt:
        def booleanize(v):
            if v in ('y', 'yes'):
                return True
            elif v in ('n', 'no'):
                return False
            raise TypeError('Unrecognized boolean placeholder: ', v)
        _continue = read_until_valid('Modify file? ', ('y', 'yes', 'n', 'no'), booleanize)
        if not _continue:
            print('Aborting version bump')
            return source
    
    # Perform the replacement
    return p.sub('\g<1>{}'.format(new_version), source)

print('Output:\n', bump(source, 'vangmeh', '1.1.0'))
print('Output:\n', bump(source, 'majqa', '9.1.0'))

Bumping version from 111.204.43 => 1.1.0
Preview:
      vangmeh:
        version: 1.1.0
Output:
 
  attributes:
    dyn:
      hostname: "zephyr24-00-dev"
      site: "dev"
    dyn_zephyr:
      env: dev
      vangmeh:
        version: 1.1.0
      majqa:
        version: 1.0.5


Bumping version from 1.0.5 => 9.1.0
Preview:
      majqa:
        version: 9.1.0
Output:
 
  attributes:
    dyn:
      hostname: "zephyr24-00-dev"
      site: "dev"
    dyn_zephyr:
      env: dev
      vangmeh:
        version: 111.204.43
      majqa:
        version: 9.1.0




In [77]:
import os
os.getenv('USER')

'jdb'